In [1]:
import pmdarima as Arima
from utils import data_handling, helpers

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import config
import pickle
import torch

# Do long and short predictions using ARIMA

For every ID in all three datasets we fit an ARIMA model and do predictions over every 96 time step window of each datasets test set.

We take the last 2000 datapoints of the train set for "full" model training for computational reasons. 
For the jumpstart evaluation we also train all ARIMA models on around 350 time steps which equals the 2 week fine-tuning horizon used in the paper.

In [2]:
# use electricity dataset
electricity_dict = data_handling.format_electricity()


for key, value in electricity_dict.items():
			electricity_dict[key]= data_handling.df_to_tensor(value)
train_standardize_dict = None

# normalize train and use matrics for val and test
electricity_dict["train"], train_standardize_dict = helpers.custom_standardizer(electricity_dict["train"])
electricity_dict["validation"], _ = helpers.custom_standardizer(electricity_dict["validation"], train_standardize_dict)
electricity_dict["test"], _ = helpers.custom_standardizer(electricity_dict["test"], train_standardize_dict)

# load bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

Length train set: 209 days, 0:00:00
Length validation set: 34 days, 0:00:00
Saving train, validation and test df for faster loading


In [3]:
def lag_tensor(df, lag):
    if lag > 0:
        return torch.cat((torch.zeros(lag, df.size(1)), df[:-lag]), dim=0)
    return df

# Example tensor of shape [2929, 348]
def create_lagged(df):

    # Lag by 24, ...
    lagged_24 = lag_tensor(df, 24)
    lagged_48 = lag_tensor(df, 24*2)
    lagged_72 = lag_tensor(df, 24*3)
    lagged_96 = lag_tensor(df, 24*4)

    length = df.size(0)
    ids = df.size(1)

    # create time of day index
    hours = torch.arange(0, 24)

    # implement sin/cosine encoding for 24h
    sin_encodings = torch.sin(2 * torch.pi * hours / 24)
    cos_encodings = torch.cos(2 * torch.pi * hours / 24)

    time_of_day_sin = sin_encodings.repeat(ids, length//23).transpose(0,1)[:length,:]
    time_of_day_cos = cos_encodings.repeat(ids, length//23).transpose(0,1)[:length,:]
    
    return torch.stack((lagged_24, lagged_48, lagged_72, lagged_96, time_of_day_sin, time_of_day_cos), dim=2)

In [4]:
lagged = create_lagged(bavaria_dict["test"])

In [5]:
def process_and_predict(df, dataset_name, data_split_description):
    num_96_horizons = int(df["test"][:,0].shape[0] / (96))
    lagged_covariates_train = create_lagged(df["train"])
    lagged_covariates_test = create_lagged(df["test"])

    #filename = config.CONFIG_OUTPUT_PATH["arima"] / f'arima_{key_}predictions.csv'
    filename = config.CONFIG_OUTPUT_PATH["arima"] / f'arima_{dataset_name}_predictions{data_split_description}.pkl'


    # Open the file and read the data
    try:
        with open(filename, 'rb') as file:
            prediction_list = pickle.load(file)
    except: 
        print("no predictions available.")
        prediction_list = []


    for id in range(len(prediction_list), df["train"].size(1)):
        model = Arima.auto_arima(df["train"][-2000:,id], exogenous=lagged_covariates_train[-2000:,id,:], stepwise=True, seasonal=True, m=24, maxiter=3)

        sum_mse = 0
        sum_mae = 0
        sum_mape = 0
        for i in range(num_96_horizons):
            time_step = i * 96
            target = df["test"][time_step : time_step+96, id]

            lagged_window_test = lagged_covariates_test[time_step:time_step+96,id,:]
            forecasts = model.predict(n_periods=96, return_conf_int=False, exogenous=lagged_window_test, alpha=0.1)

            sum_mse += mean_squared_error(forecasts, target)
            sum_mae += mean_absolute_error(forecasts, target)
            sum_mape = mean_absolute_percentage_error(forecasts, target)
            

        prediction_list.append([sum_mse / num_96_horizons, sum_mae / num_96_horizons, sum_mape / num_96_horizons])
        print(f"MSE of {id}: ", sum_mse / num_96_horizons)


        # save as pickle
        with open(filename, 'wb') as file:
            pickle.dump(prediction_list, file)

In [6]:
# do training on 2000 time steps
#process_and_predict(electricity_dict, "electricity", "full")
#process_and_predict(bavaria_dict, "bavaria", "full")
process_and_predict(gp_dict, "genome_project", "full")

MSE of 107:  0.7257356199801197
MSE of 108:  0.36350640998175093
MSE of 109:  1.4969255229323815
MSE of 110:  0.52440694424908
MSE of 111:  0.7154264644763294
MSE of 112:  0.7747190211010762


KeyboardInterrupt: 